In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from typing import Optional, Callable
import os

from scripts.loops import train_loop, valid_loop, test_loop
from scripts.dataset_template import DatasetTemplate
from scripts.models.autoencoder_cnn import CNNAutoencoder
from scripts.models.autoencoder_lin import LinearAutoencoder
from scripts.models.autoencoder_lstm import LSTMAutoencoder

In [2]:
#check if cuda available

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [3]:
#paths
data_train_path = "/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/Data/Data_channels/train/comeretial"
data_test_folder_path = "/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/Data/Data_channels/test"
data_valid_folder_path = "/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/Data/Data_channels/valid"

testing_folders = os.listdir(data_test_folder_path)
valid_folders = os.listdir(data_valid_folder_path)

In [4]:
#hyperparameters
batch_size = 1
learning_rate = 0.01
epochs = 1

In [5]:
#transforms
data_transforms = transforms.Compose([transforms.ToTensor()])

In [6]:
#datasets and dataloaders

#train
train_dataset = DatasetTemplate(data_train_path, 0, transform = data_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#valid
valid_datasets = {}
valid_dataloaders = {}

for valid_folder_name in valid_folders:
    valid_folder_path_all = os.path.join(data_valid_folder_path, valid_folder_name)
    valid_datasets[valid_folder_name] = DatasetTemplate(valid_folder_path_all, 1)
    valid_dataloaders[valid_folder_name] = DataLoader(valid_datasets[valid_folder_name], batch_size=batch_size, shuffle=True)

#test
test_datasets = {}
test_dataloaders = {}

for test_folder_name in testing_folders:
    test_folder_path_all = os.path.join(data_test_folder_path, test_folder_name)
    test_datasets[test_folder_name] = DatasetTemplate(test_folder_path_all, 1)
    test_dataloaders[test_folder_name] = DataLoader(test_datasets[test_folder_name], batch_size=batch_size, shuffle=True)

In [7]:
valid_dataloaders

{'Fake_Bts_PCI_466_wPA': <torch.utils.data.dataloader.DataLoader at 0x7f25600a21d0>,
 'Fake_Bts_PCI_466': <torch.utils.data.dataloader.DataLoader at 0x7f25600a04f0>}

In [8]:
#define model
model = CNNAutoencoder().to(device)
criterion = nn.MSELoss()
loss_RMS = lambda x,y: torch.sqrt(criterion(x, y))
metric_fn_RMS = lambda x,y: torch.sqrt(criterion(x, y))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


valid_loop(test_dataloaders[list(test_dataloaders.keys())[0]],
          model,
          loss_RMS,
          metric_fn_RMS,
          device=device)

valid Error: 
 Accuracy: 327.4010207312448%, Avg loss: 327.401021 



(327.4010207312448, 327.4010207312448)

In [9]:
#epochs

# Lists to log the losses during the training process
train_losses = []
train_metrics = []

valid_losses_true = []
valid_metrics_true = []

valid_losses_false = []
valid_metrics_false = []

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_loss, train_metric = train_loop(train_dataloader,
                                     model,
                                     loss_RMS,
                                     optimizer)
    
  valid_loss_true, valid_metric_true = valid_loop(valid_dataloaders['Fake_Bts_PCI_466'],
                                                  model,
                                                  loss_RMS,
                                                  metric_fn_RMS)

  valid_loss_false, valid_metric_false = valid_loop(valid_dataloaders['Fake_Bts_PCI_466_wPA'],
                                                    model,
                                                    loss_RMS,
                                                    metric_fn_RMS)
    

  train_losses.append(train_loss)
  train_metrics.append(train_metric)

  valid_losses_true.append(valid_loss_true)
  valid_metrics_true.append(valid_metric_true)

  valid_losses_false.append(valid_loss_false)
  valid_metrics_false.append(valid_metric_false)
    

print("Done!")

Epoch 1
-------------------------------
loss: 171.045486  [    0/81710]

loss: 94.782730  [10000/81710]

loss: 85.276962  [20000/81710]

loss: 327.691193  [30000/81710]

loss: 101.895264  [40000/81710]

loss: 74.863098  [50000/81710]

loss: 88.736679  [60000/81710]

loss: 100.333397  [70000/81710]

loss: 248.940872  [80000/81710]

valid Error: 
 Accuracy: 708.2602741622925%, Avg loss: 708.260274 

valid Error: 
 Accuracy: 563.1163077139639%, Avg loss: 563.116308 

Done!


In [10]:
#testing

for testing_name, testing_loader in test_dataloaders.items():
    test_losses, test_metrics = test_loop(testing_loader, 
                                            model,
                                            loss_RMS,
                                             metric_fn_RMS)
    mean_loss = np.mean(np.array(test_losses))
    mean_metrics = np.mean(np.array(test_metrics))
    print(f"Dataset name: {testing_name}, mean_loss: {mean_loss}, mean_acc: {mean_metrics}")

Dataset name: Fake_Bts_PCI_466_wPA_traffic, mean_loss: 327.395322468816, mean_acc: 327.395322468816
Dataset name: Fake_Bts_PCI_12, mean_loss: 448.14388313940015, mean_acc: 448.14388313940015
Dataset name: Fake_Bts_PCI_466_traffic, mean_loss: 922.3986207925656, mean_acc: 922.3986207925656
Dataset name: Fake_Bts_PCI_12_wPA, mean_loss: 537.0704357122285, mean_acc: 537.0704357122285
